In [1]:
import pandas as pd
import mysql.connector as mc
import matplotlib
import calendar

In [2]:
db = mc.connect(
  host='172.18.55.6',
  user='comandato',
  passwd='comandato123',
)
cur= db.cursor()

In [3]:
Dia='16'
Mes='05'
Anio='2019'
fecha_pago=Dia+Mes+Anio

In [4]:
F_Mes='MAYO'

# OBTENER GESTION DE ASESOR

In [5]:
querry_gestion="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.GESTION where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00';""".format(Anio,Mes,Dia)
gestion=pd.read_sql(querry_gestion, con=db)

In [6]:
gestion.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0104772363,D013113,2,NO CONTACTO,HACER LLAMADA,NO CONTESTA,0984386351
1,0917597551,D013110,2,NO CONTACTO,HACER LLAMADA,BUZON DE MENSAJES,0991138842
2,0916719172,D013113,2,TITULAR,HACER LLAMADA,VOLVER A LLAMAR,0979811719
3,0911637395,D013113,2,NO CONTACTO,HACER LLAMADA,NO CONTESTA,0999508037
4,0909622037,D013110,2,TITULAR,HACER LLAMADA,ACUERDO DE PAGO,0999113107


In [7]:
gestion_asesor=gestion[(gestion['ACCION_REALIZADA']=='HACER LLAMADA') & ((gestion['CONTACTO_GESTIONO']=='TITULAR') | (gestion['CONTACTO_GESTIONO']=='TERC. VALIDO'))]

In [8]:
gestion_asesor=gestion_asesor.sort_values(by=['FECHA_GESTION'],ascending=[True])
gestion_asesor=gestion_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')
gestion_asesor = gestion_asesor.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})

# OBTENER GESTION MASIVOS

In [9]:
querry_masivos="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.gestion_ivr where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00';""".format(Anio,Mes,Dia)
gestion_masivos=pd.read_sql(querry_masivos, con=db)

In [10]:
gestion_masivos.tail()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
67647,0924932684,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996793546
67648,0928443639,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0985184760
67649,0929222313,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,042378488
67650,0931070072,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996815947
67651,0940393309,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996340027


In [11]:
gestion_masivos=gestion_masivos[gestion_masivos['RESPUESTA_OBTENIDA']!='NO CONTESTA IVR']

In [12]:
gestion_masivos=gestion_masivos.sort_values(by=['FECHA_GESTION'],ascending=[True])
gestion_masivos=gestion_masivos.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')
gestion_masivos = gestion_masivos.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})
gestion_masivos['CONTACTO_GESTIONO']=gestion_masivos['ACCION_REALIZADA']
gestion_masivos['ACCION_REALIZADA']='GESTION MASIVOS'

# TOTAL GESTIONES PRODUCTIVAS

In [13]:
gestion_total= pd.concat([gestion_asesor,gestion_masivos])
# gestion_total=gestion_total.sort_values(by=['FECHA_GESTION'],ascending=[True])
gestion_total=gestion_total.drop_duplicates(['IDENTIFICACION'], keep='first')

# MULTAS ASIGNADAS

In [14]:
multas="""SELECT *   FROM SISTEMECUADOR_ATM.MULTAS_MENSUALES where Mes={0}""".format(Mes)
resultado_multas=pd.read_sql(multas, con=db)

In [15]:
resultado_multas.head()

,Cedula,No_multa,Anio,Mes,Dia,Dia_retiro,Dias_mora,Saldo,Cartera,Segmento,Descripcion,Placa,Pago,dia_pago,tipo
0,00042448K,5722394,2019,5,1,None,191.0,115.80,CORRIENTE,MULTAS NUEVAS,COND.EXCEDA DENTRO RANGO MODERADO LÍMITES DE V...,GRP0269,None,None,None
1,1000005262,4854542,2018,5,2,None,52.0,75.86,CONVENIO,CUOTA,Atraso,nan,None,None,None
2,1000041952,4087528,2018,5,2,None,232.0,112.50,CORRIENTE,FASE II,COND.EXCEDA DENTRO RANGO MODERADO LÍMITES DE V...,ABA1143,None,None,None
3,1000131977,1786073,2018,5,2,None,737.0,54.90,CORRIENTE,MENOR CUANTIA,DETEN.ESTAC.VEH.LUGAR NO PERMITIDO DEJAR/RECIB...,PAA6834,None,None,None
4,1000203719,5039556,2018,5,2,None,45.0,64.55,CONVENIO,CUOTA,Atraso,nan,None,None,None


In [16]:
dias_mora_multa=resultado_multas[['No_multa','Dias_mora']].copy()

In [17]:
dias_mora_multa = dias_mora_multa.rename(columns={'No_multa':'ID_FACTURA','Dias_mora':'DIAS_MORA'})
dias_mora_multa['ID_FACTURA']=dias_mora_multa['ID_FACTURA'].apply(int)

In [18]:
dias_mora_multa.head()

,ID_FACTURA,DIAS_MORA
0,5722394,191.0
1,4854542,52.0
2,4087528,232.0
3,1786073,737.0
4,5039556,45.0


# PAGOS

In [19]:
pago_corr={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CORRIENTE',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION'
}

pago_cuot={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CUOTACERO',
    'cartera':'CUOTA',
    'identificador':'ID'
}

pago_conv={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CONVENIOS',
    'cartera':'CONVENIO',
    'identificador':'ID'
}

In [20]:
pagos_corr=pd.read_excel(open(pago_corr['ruta'],'rb'), sheet_name=pago_corr['hoja'], dtype={pago_corr['identificador']: str})
pagos_cuot=pd.read_excel(open(pago_cuot['ruta'],'rb'), sheet_name=pago_cuot['hoja'], dtype={pago_cuot['identificador']: str})
pagos_conv=pd.read_excel(open(pago_conv['ruta'],'rb'), sheet_name=pago_conv['hoja'], dtype={pago_conv['identificador']: str})

In [21]:
def agregar_ceros(base):
    ced=base['IDENTIFICACION']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

### CORRIENTE

In [22]:
pagos_corr['IDENTIFICACION']=pagos_corr.apply(agregar_ceros,axis=1)

In [23]:
pagos_corr['DIA_PAGO'] = pagos_corr['FECHA_PAGO'].dt.day
pagos_corr = pagos_corr.rename(columns={'CASA COBRO':'CASA_COBRO'})

In [24]:
pagos_corr['TIPO']='CORRIENTE'
cruce_pag_corr=pd.merge(pagos_corr,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_corr=cruce_pag_corr[['IDENTIFICACION','ID_FACTURA','VALOR_INFRACCION','FECHA_PAGO','DIA_PAGO','TIPO','DIAS_MORA']]
cruce_pag_corr.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0907964464,6347039,115.8,2019-05-01,1,CORRIENTE,172.0
1,1711723831,5112985,19.3,2019-05-01,1,CORRIENTE,326.0
2,1304396581,5063205,19.3,2019-05-01,1,CORRIENTE,335.0
3,1304396581,5055893,19.3,2019-05-01,1,CORRIENTE,337.0
4,0912125002,6382875,19.7,2019-05-01,1,CORRIENTE,32.0


## CUOTA CERO

In [25]:
def identificar_pago(base):
    valor1 = base['VALOR_INFRACCION']
    valor2 = base['VALOR_PAGO_CAPITAL']
    mora = base['DIAS_MORA']
    if mora<=180:
        return valor2
    return valor1

In [26]:
pagos_cuot = pagos_cuot.rename(columns={'ID':'IDENTIFICACION'})

In [27]:
pagos_cuot['IDENTIFICACION']=pagos_cuot.apply(agregar_ceros,axis=1)

In [28]:
pagos_cuot['TIPO']='CUOTA CERO'
pagos_cuot['DIA_PAGO'] = pagos_cuot['FECHA_REGISTRO_PAGO'].dt.day
pagos_cuot['MONTO_PAGO']=pagos_cuot.apply(identificar_pago,axis=1)
pagos_cuot_resumen=pagos_cuot[['IDENTIFICACION','ID_FACTURA_INICIAL','MONTO_PAGO','FECHA_REGISTRO_PAGO','DIA_PAGO','TIPO']]
pagos_cuot_resumen = pagos_cuot_resumen.rename(columns={'ID_FACTURA_INICIAL':'ID_FACTURA','MONTO_PAGO':'VALOR_INFRACCION','FECHA_REGISTRO_PAGO':'FECHA_PAGO'})
cruce_pag_cuot=pd.merge(pagos_cuot_resumen,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_cuot.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0919709139,5533365,19.30,2019-05-01,1,CUOTA CERO,236.0
1,0919709139,5608800,386.00,2019-05-01,1,CUOTA CERO,218.0
2,0919709139,5973029,112.04,2019-05-01,1,CUOTA CERO,128.0
3,0918099920,6348438,90.64,2019-05-01,1,CUOTA CERO,40.0
4,0919709139,5358553,19.30,2019-05-01,1,CUOTA CERO,273.0


# CONVENIO

In [29]:
pagos_conv = pagos_conv.rename(columns={'ID':'IDENTIFICACION'})

In [30]:
pagos_conv['IDENTIFICACION']=pagos_conv.apply(agregar_ceros,axis=1)

In [31]:
pagos_conv['TIPO']='CONVENIO'
pagos_conv['DIA_PAGO'] = pagos_conv['FECHA_PAGO'].dt.day

In [32]:
pagos_conv_resumen=pagos_conv[['IDENTIFICACION','ID_CONVENIO','VALOR_PAGAR_CUOTA','FECHA_PAGO','DIA_PAGO','TIPO','NRO_CUOTA_EMISION']]
cruce_pag_conv = pagos_conv_resumen.rename(columns={'ID_CONVENIO':'ID_FACTURA','VALOR_PAGAR_CUOTA':'VALOR_INFRACCION','NRO_CUOTA_EMISION':'DIAS_MORA'})

In [33]:
cruce_pag_conv.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0922469549,306207,102.93,2019-05-01,1,CONVENIO,6
1,0922469549,306207,102.93,2019-05-01,1,CONVENIO,2
2,0922469549,306207,102.93,2019-05-01,1,CONVENIO,3
3,0941443665,350631,30.88,2019-05-01,1,CONVENIO,1
4,0941443665,350631,30.88,2019-05-01,1,CONVENIO,2


## UNIR LOS PAGOS

In [34]:
pagos=[cruce_pag_cuot,cruce_pag_corr,cruce_pag_conv]
total_pagos=pd.concat(pagos)

In [35]:
total_pagos['TIPO'].value_counts()

CORRIENTE     1019
CONVENIO       486
CUOTA CERO     344
Name: TIPO, dtype: int64

## CRUCE CON GESTIONES

In [36]:
cruce_gestion_total=pd.merge(total_pagos,gestion_total,on=['IDENTIFICACION'],how='left')
cruce_gestion_total.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0919709139,5533365,19.30,2019-05-01,1,CUOTA CERO,236.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0919709139,5608800,386.00,2019-05-01,1,CUOTA CERO,218.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0919709139,5973029,112.04,2019-05-01,1,CUOTA CERO,128.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0918099920,6348438,90.64,2019-05-01,1,CUOTA CERO,40.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0919709139,5358553,19.30,2019-05-01,1,CUOTA CERO,273.0,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
cruce_gestion_total['ACCION_REALIZADA'].value_counts()

GESTION MASIVOS    955
HACER LLAMADA      519
Name: ACCION_REALIZADA, dtype: int64

# Honorarios

In [38]:
def altura_mora(base):
    altura = base['DIAS_MORA']
    if  altura>=0 and altura<11:
        return 'DE 0 A 10'
    if  altura>=11 and altura<61:
        return 'DE 11 A 60'
    if  altura>=61 and altura<181:
        return 'DE 61 A 180'
    if  altura>=181 and altura<361:
        return 'DE 181 A 361'
    if  altura>=361:
        return 'MAYOR A 361'
    else:
        return 'None'

In [39]:
cruce_gestion_total['Altura_Mora']=cruce_gestion_total.apply(altura_mora,axis=1)

In [40]:
cruce_gestion_total['Altura_Mora'].value_counts()

DE 0 A 10       465
DE 11 A 60      460
DE 181 A 361    391
DE 61 A 180     322
MAYOR A 361     211
Name: Altura_Mora, dtype: int64

In [41]:
def honorarios_porct(base):
    tipo = base['TIPO']
    accion = base['ACCION_REALIZADA']
    altura_mora = base['Altura_Mora']
    porct = 0.7 if accion=='GESTION MASIVOS' else 1
    if tipo == 'CORRIENTE' or tipo == 'CONVENIO':
        if altura_mora=='DE 0 A 10':
            return porct*0
        if altura_mora=='DE 11 A 60':
            return porct*0.02
        if altura_mora=='DE 61 A 180':
            return porct*0.04
        if altura_mora=='DE 181 A 361':
            return porct*0.05
        if altura_mora=='MAYOR A 361':
            return porct*0.06
    elif tipo == 'CUOTA CERO':
        if altura_mora=='DE 0 A 10':
            return 0
        if altura_mora=='DE 11 A 60':
            return 0.02
        if altura_mora=='DE 61 A 180':
            return 0.04
        if altura_mora=='DE 181 A 361':
            return 0.039
        if altura_mora=='MAYOR A 361':
            return 0.039
    else:
        return 0

In [42]:
cruce_gestion_total['HONORARIOS_PORCENTAJE']=cruce_gestion_total.apply(honorarios_porct,axis=1)
cruce_gestion_total['HONORARIOS']=cruce_gestion_total['HONORARIOS_PORCENTAJE']*cruce_gestion_total['VALOR_INFRACCION']

In [43]:
cruce_gestion_total['TIPO'].value_counts()

CORRIENTE     1019
CONVENIO       486
CUOTA CERO     344
Name: TIPO, dtype: int64

In [44]:
def validar(base):
    dia_pago=base['DIA_PAGO']
    dia_gestion=base['FECHA_GESTION']
    if dia_gestion<=dia_pago:
        return 'SI'
    return 'NO'

In [45]:
cruce_gestion_total['VALIDO']=cruce_gestion_total.apply(validar,axis=1)

In [46]:
cruce_gestion_total['TIPO'].value_counts()

CORRIENTE     1019
CONVENIO       486
CUOTA CERO     344
Name: TIPO, dtype: int64

## HONORARIOS VALIDOS

In [47]:
print(cruce_gestion_total[cruce_gestion_total['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(cruce_gestion_total[cruce_gestion_total['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

            VALOR_INFRACCION  HONORARIOS
TIPO                                    
CONVENIO            19359.19    24.84996
CORRIENTE           63129.65  1679.71800
CUOTA CERO          25141.90   828.13570
VALOR_INFRACCION    107630.74000
HONORARIOS            2532.70366
dtype: float64


In [48]:
cruce_gestion_total[cruce_gestion_total['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).plot.line()

## HONORARIOS TOTALES

In [49]:
print(cruce_gestion_total.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(cruce_gestion_total.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

            VALOR_INFRACCION  HONORARIOS
TIPO                                    
CONVENIO            31985.27    31.88236
CORRIENTE           81017.95  2374.73060
CUOTA CERO          30593.03  1011.41419
VALOR_INFRACCION    143596.25000
HONORARIOS            3418.02715
dtype: float64


## HONORARIOS NO VALIDOS

In [50]:
print(cruce_gestion_total[cruce_gestion_total['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(cruce_gestion_total[cruce_gestion_total['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

            VALOR_INFRACCION  HONORARIOS
TIPO                                    
CONVENIO            12626.08     7.03240
CORRIENTE           17888.30   695.01260
CUOTA CERO           5451.13   183.27849
VALOR_INFRACCION    35965.51000
HONORARIOS            885.32349
dtype: float64


In [51]:
nuevo_archivo='ANALISIS DE PAGOS CON PREORIDAD FECHA.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
cruce_gestion_total.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()

In [52]:
nuevo_archivo='GESTION ASESOR.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
gestion_asesor.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()